In [1]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
import os
import cv2
from PIL import ImageGrab
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from ipynb.fs.full.config import getItem_string
from ipynb.fs.full.config import getItem_int
from ipynb.fs.full.Util import dictToTxt,TxtToDict
from ipynb.fs.full.preprocessing_images import generate_train_set,generate_test_set,loadImage

from collections import Counter

#load dataset for training
#return all images in the dataset & corresponding name labels
def load_all_dataset(path):
    images=[]
    labels=[] 
    dictionary={}
    images,labels = loadImage(path)  
    #convert images to 4d array  (number_of_image, size_of_image,size_of_image,3)   3 means RGB
    images = np.array(images)
#     print(images.shape) 
    #label all images with corresponding names(numbers) 
    res_labels=[]
    before_name=""
    number=0
    for label in range(len(labels)):
        #if it is the first image
        if label==0:
            res_labels.append(number)
            before_name=labels[label]
            dictionary.update({str(number):labels[label]})
        else:
            #if last name is same as current one, append same number to the label array
            if before_name==labels[label]:
                res_labels.append(number)
            else:#last name is different from current one, meaning an image from other person)
                number+=1
                res_labels.append(number) 
                before_name=labels[label] 
                dictionary.update({str(number):labels[label]})
#             print(images)
#             print(res_labels)
    
    dictToTxt(dictionary,"names.txt") 
    return images, res_labels
    

def load_training_dataset(path):
    images=[]
    labels=[] 
    dictionary={}
    images,labels = generate_train_set()
    #convert images to 4d array  (number_of_image, size_of_image,size_of_image,3)   3 means RGB
    images = np.array(images)
#     print(images.shape) 
    #label all images with corresponding names(numbers) 
    res_labels=[]
    before_name=""
    number=0
    for label in range(len(labels)):
        #if it is the first image
        if label==0:
            res_labels.append(number)
            before_name=labels[label]
            dictionary.update({str(number):labels[label]})
        else:
            #if last name is same as current one, append same number to the label array
            if before_name==labels[label]:
                res_labels.append(number)
            else:#last name is different from current one, meaning an image from other person)
                number+=1
                res_labels.append(number) 
                before_name=labels[label] 
                dictionary.update({str(number):labels[label]})
#             print(images)
#             print(res_labels)
    
    dictToTxt(dictionary,"train_names.txt") 
    return images, res_labels    

def load_test_dataset(path):
    images=[]
    labels=[] 
    dictionary={}
    images,labels = generate_test_set()
    print(len(images))
    #convert images to 4d array  (number_of_image, size_of_image,size_of_image,3)   3 means RGB
    images = np.array(images)
#     print(images.shape) 
    #label all images with corresponding names(numbers) 
    res_labels=[]
    before_name=""
    number=0
    for label in range(len(labels)):
        #if it is the first image
        if label==0:
            res_labels.append(number)
            before_name=labels[label]
            dictionary.update({str(number):labels[label]})
        else:
            #if last name is same as current one, append same number to the label array
            if before_name==labels[label]:
                res_labels.append(number)
            else:#last name is different from current one, meaning an image from other person)
                number+=1
                res_labels.append(number) 
                before_name=labels[label] 
                dictionary.update({str(number):labels[label]})
#             print(images)
#             print(res_labels)
    
    dictToTxt(dictionary,"test_names.txt") 
    return images, res_labels   

#load training set, test set
def load(dataset):
    size = getItem_int("dataset","image_size")
    numberOfClass = getItem_int("dataset","number_of_class")
    #RGB has 3 channels
    channel = 3

    images,labels = load_all_dataset(getItem_string("dataset","test_dataset_path"))
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.3, random_state = random.randint(0, 100))                 
                                        #number of images,     size,size,   channel)
    train_images = train_images.reshape(train_images.shape[0], size, size, channel)
    test_images = test_images.reshape(test_images.shape[0], size, size, channel)            

    print(train_images.shape[0], 'training set')
    print(test_images.shape[0], 'test set')                            
    train_labels = np_utils.to_categorical(train_labels, numberOfClass)                        
    test_labels = np_utils.to_categorical(test_labels, numberOfClass)   
    
    #normalize
    train_images = train_images.astype('float32')    
    test_images  = test_images.astype('float32')
    
    train_images/=255
    test_images/=255
    

    
    return train_images, test_images, train_labels, test_labels

                    
        
def build_model():
    IMAGE_SIZE=getItem_int("dataset","image_size")
    numberOfClass =  getItem_int("dataset","number_of_class")
    model = Sequential([
        Conv2D(64, (3, 3),activation='relu',padding='same',input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)),
        Conv2D(64, (3, 3),activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),       
        Conv2D(64, (3, 3),activation='relu',padding='same'),      
        Conv2D(64, (3, 3),activation='relu',padding='same'),
        MaxPooling2D(2, 2),
        Dropout(0.25),        
        Flatten(),
        Dense(512,activation='relu'),
        Dense(numberOfClass,activation='softmax')
    ])
    #print network
    model.summary()
    return model


    #train the model
def train(model, epochs,train_images, train_labels):        
    sgd = SGD() 
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])  
    #add variations
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        zca_whitening = False,
        rotation_range=270,
        width_shift_range=0.2,
        height_shift_range=0.2)
    datagen.fit(train_images) 
   
    model.fit_generator(datagen.flow(train_images, train_labels,batch_size=20),
                             steps_per_epoch=len(train_images) / 20,
                             epochs = epochs) 
    print("training done")
    return model
    
   
def saveModel(model, path):  
    print("save model")
    model.save(path)
    print("saved successfully")

def loadModel(path):  
    print("load model")
    return load_model(path)
    
 
def evaluate(model, test_images, test_labels):
    #open progress bar
    score = model.evaluate(test_images, test_labels, verbose = 1)
#          print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))
    print("loss: ",score[0], "accuracy: ",score[1] * 100,"%")

    #predict face
def face_predict(model,image):  
    IMAGE_SIZE=getItem_int("dataset","image_size")
    image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))     
    image = image.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 3))                    

    #normalize
    image= image.astype('float32')
    image/=255
    #between 0-1
    prob = model.predict_proba(image)#[[1,2,3,4]]
    print('result-------', prob)

    prob = prob[0].tolist()
    maxValue = max(prob)
    print(maxValue)
    maxIndex = prob.index(maxValue) 
    #return the result
    if(maxValue>0.9):
        return maxIndex
    else:
        return -1
    
def resizeWindow(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    return cv2.resize(image, dim, interpolation=inter) 

#image(require a imgpath) or video or camera 
def test(source,path):
    #load the model
    model = loadModel(getItem_string("dataset","model_path"))    
    #get names
    dictionary = TxtToDict("names.txt")
    if(source=="camera"):
        cap = cv2.VideoCapture(0)
        cv2.namedWindow('Camera')
        while True:
            #read a frame
            ret, frame = cap.read()      
            if ret is True: 
                #convert into greyscale
                grayscale_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #create a cascade classifier
            face_cascade = cv2.CascadeClassifier(getItem_string("opencv","cascadeclassifier")  )                

            #detect faces 
            faceRects = face_cascade.detectMultiScale(grayscale_image,scaleFactor = 1.2,minNeighbors = 3)  
            personId=-1
            #if any face is detected
            if len(faceRects) > 0:                 
                for (x,y,w,h) in faceRects: 
                    #draw rectangles contained faces
                    frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
                    ##crop images, make the area bigger to improve the rate
                    image = frame[y-10: y+h+10, x-10: x+w+10]
                    personId = face_predict(model,image)   

               #no one detected                       
                if(personId== -1):        
                    cv2.putText(frame,                   #window
                                "unknown",               #name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight
                else:
                    cv2.putText(frame,                   #window
                                dictionary[str(personId)],#name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight
            else:
                continue                   
            cv2.imshow("Camera", frame)
            k = cv2.waitKey(10)  
            #press q to exit
            if k & 0xFF == ord('q'):
                break
        cap.release()
                
    elif(source=='video'):                
        while True:
            #get a specfic area on the screen
            screenshot =  ImageGrab.grab(bbox=(40,290,890,770))
            #reshape the image
            screenshot_numpy =   np.array(screenshot.getdata(),dtype='uint8')\
            .reshape((screenshot.size[1],screenshot.size[0],3))
            #resize window
            shrinkedWindow = resizeWindow(screenshot_numpy,650) 
            #convert into greyscale
            grayscale_image = cv2.cvtColor(shrinkedWindow, cv2.COLOR_BGR2GRAY)
            #create a cascade classifier
            face_cascade = cv2.CascadeClassifier(getItem_string("opencv","cascadeclassifier"))                
            #detect faces
            faceRects = face_cascade.detectMultiScale(grayscale_image,scaleFactor = 1.2,minNeighbors = 3)   
            #if any face is detected
            if len(faceRects) > 0:                 
                for (x,y,w,h) in faceRects: 
                    #draw rectangles contained faces
                    shrinkedWindow = cv2.rectangle(shrinkedWindow,(x,y),(x+w,y+h),(255,0,0),2)
                    ##crop images, make the area bigger to improve the success rate
                    image = shrinkedWindow[y-10: y+h+10, x-10: x+w+10]
                    #predict
                    personId = face_predict(model,image)   

                #no one detected                       
                if(personId== -1):        
                    cv2.putText(shrinkedWindow,          #window
                                "unknown",               #name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight
                else:
                    cv2.putText(shrinkedWindow,          #window
                                dictionary[str(personId)],#name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight

            else:
                continue        

            cv2.imshow('video',cv2.cvtColor(shrinkedWindow, cv2.COLOR_BGR2RGB))

            k = cv2.waitKey(20)
            #press q to exit
            if k & 0xFF == ord('q'):
                break
    elif(source=='image'):
        while True:       
            img = cv2.imread(path)
            img = cv2.resize(img,(400,400))
            #convert into greyscale
            grayscale_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #create a cascade classifier
            face_cascade = cv2.CascadeClassifier(getItem_string("opencv","cascadeclassifier"))                
            #detect faces
            faceRects = face_cascade.detectMultiScale(grayscale_image,scaleFactor = 1.2,minNeighbors = 3)   
            #if any face is detected
            if len(faceRects) > 0:                 
                for (x,y,w,h) in faceRects: 
                    #draw rectangles contained faces
                    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                    ##crop images, make the area bigger to improve the success rate
                    image = img[y-10: y+h+10, x-10: x+w+10]
                    #predict
                    personId = face_predict(model,image)   

                #no one detected                       
                if(personId== -1):        
                    cv2.putText(img,          #window
                                "unknown",               #name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight
                else:
                    cv2.putText(img,          #window
                                dictionary[str(personId)],#name
                                (x,y),                   #coordinate
                                cv2.FONT_HERSHEY_SIMPLEX,#font
                                1,                       #font size
                                255,                     #color
                                2)                       #stroke weight

            else:
                continue        
            
            cv2.imshow("image",img)
            
            k = cv2.waitKey(10)
            #press q to exit
            if k & 0xFF == ord('q'):
                break
    cv2.destroyAllWindows()

   
    
# if __name__ == '__main__':
#     test("image","E:/project/small_face_dataset/Aaron_Eckhart/Aaron_Eckhart_20.jpg")

#     train_images, test_images, train_labels, test_labels = load(getItem_string("dataset","test_dataset_path"))   
#     model = loadModel(getItem_string("dataset","model_path"))    
#     evaluate(model, test_images, test_labels)

#     train_images, test_images, train_labels, test_labels = load(getItem_string("dataset","test_dataset_path"))   
#     model = build_model()
#     model = train(model,10,train_images, train_labels)
#     saveModel(model, getItem_string("dataset","model_path"))

   
    


Using TensorFlow backend.


images are loaded!
3150 training set
1350 test set
load model
1350/1350 [==============================] - 11s 8ms/step
loss:  0.02758220835485392 accuracy:  99.18518662452698 %
